<a href="https://colab.research.google.com/github/nikhil-mathews/MastersPr_Predicting-Human-Pathogen-PPIs-using-Natural-Language-Processing-methods/blob/main/3.Attention%20%26%20Transformers/Transformers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

try:
  import pickle5 as pickle
except:
  !pip install pickle5
  import pickle5 as pickle
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, GlobalMaxPooling1D,GlobalAveragePooling1D
from keras.layers import Conv1D, MaxPooling1D, Embedding, Flatten,Attention
from keras.models import Model
from sklearn.metrics import roc_auc_score,roc_curve, auc
from numpy import random
from keras.layers import LSTM, Bidirectional, GlobalMaxPool1D, Dropout
from keras.optimizers import Adam
from keras.utils.vis_utils import plot_model

import sys
sys.path.insert(0,'/content/drive/MyDrive/ML_Data/')
import functions as f

def load_data(D=1,randomize=False):
    try:
        with open('/content/drive/MyDrive/ML_Data/df_train_'+str(D)+'D.pickle', 'rb') as handle:
            df_train = pickle.load(handle)
    except:
        df_train = pd.read_pickle("C:/Users/nik00/py/proj/hyppi-train.pkl")
    try:
        with open('/content/drive/MyDrive/ML_Data/df_test_'+str(D)+'D.pickle', 'rb') as handle:
            df_test = pickle.load(handle)
    except:
        df_test = pd.read_pickle("C:/Users/nik00/py/proj/hyppi-independent.pkl")
    if randomize:
        return shuff_together(df_train,df_test)
    else:
        return df_train,df_test
# df_train,df_test = load_data()
# print('The data used will be:')
# df_train[['Joined']]

     |████████████████████████████████| 133kB 11.2MB/s 
  Created wheel for pickle5: filename=pickle5-0.0.11-cp37-cp37m-linux_x86_64.whl size=219235 sha256=352ca99ea48d23f29fc24857cb107f41367610e30a5094c03ea809412dfc885f
  Stored in directory: /root/.cache/pip/wheels/a6/90/95/f889ca4aa8b0e0c7f21c8470b6f5d6032f0390a3a141a9a3bd
Successfully built pickle5


In [ ]:
df_train,df_test = load_data(1)
print('The data used will be:')
df_train[['Joined']]

The data used will be:


,Joined
0,"[M, K, D, K, Q, K, K, K, K, E, R, T, W, A, E, ..."
1,"[M, S, W, S, G, L, L, H, G, L, N, T, S, L, T, ..."
2,"[M, S, L, F, D, L, F, R, G, F, F, G, F, P, G, ..."
3,"[M, A, V, A, P, R, L, F, G, G, L, C, F, R, F, ..."
4,"[M, S, T, I, Q, S, E, T, D, C, Y, D, I, I, E, ..."
...,...
6265,"[M, S, Y, F, G, E, H, F, W, G, E, K, N, H, G, ..."
6266,"[M, T, V, G, K, S, S, K, M, L, Q, H, I, D, Y, ..."
6267,"[M, N, N, L, S, F, S, E, L, C, C, L, F, C, C, ..."
6268,"[M, A, P, E, I, N, L, P, G, P, M, S, L, I, D, ..."


In [ ]:
data_join,data_test_join,num_words_join,MAX_SEQUENCE_LENGTH_,MAX_VOCAB_SIZE_ = f.get_seq_data_join(5000,2000,df_train,df_test,pad='pre')

MAX_VOCAB_SIZE is 5000
MAX_SEQUENCE_LENGTH is 2000
max sequence_data length: 8966
min sequence_data length: 107
median sequence_data length: 881
max word index: 20
Found 20 unique tokens.
pre padding.
Shape of data tensor: (6270, 2000)
max sequences_test length: 6126
min sequences_test length: 134
median sequences_test length: 901
pre padding for test seq.
Shape of data_test tensor: (1514, 2000)
num_words is 21


In [ ]:
DROP = 0.4
BATCH_SIZE = 64
EPOCHS = 10
ip = f.transf_model(MAX_SEQUENCE_LENGTH_,num_words_join,5,DROP)
x = Dropout(DROP)(ip.output)
x = Dense(128, activation="relu")(x)
x = Dropout(DROP)(x)
outputs = Dense(1, activation="sigmoid")(x)
model1D_CNN_join=Model(ip.input,outputs)

model1D_CNN_join.compile(loss='binary_crossentropy', optimizer=Adam(lr=0.01), metrics=['accuracy'])
model1D_CNN_join.fit(data_join, df_train['label'].values,batch_size= BATCH_SIZE,epochs=EPOCHS, validation_data=(data_test_join,df_test['label'].values))
print(roc_auc_score(df_test['label'].values, model1D_CNN_join.predict(data_test_join)))

Epoch 1/10
98/98 [==============================] - 22s 209ms/step - loss: 0.6859 - accuracy: 0.5674 - val_loss: 0.6561 - val_accuracy: 0.6189
Epoch 2/10
98/98 [==============================] - 20s 205ms/step - loss: 0.6576 - accuracy: 0.6209 - val_loss: 0.6204 - val_accuracy: 0.6678
Epoch 3/10
98/98 [==============================] - 20s 205ms/step - loss: 0.6496 - accuracy: 0.6290 - val_loss: 0.6204 - val_accuracy: 0.6691
Epoch 4/10
98/98 [==============================] - 20s 204ms/step - loss: 0.6443 - accuracy: 0.6383 - val_loss: 0.6257 - val_accuracy: 0.6764
Epoch 5/10
98/98 [==============================] - 20s 205ms/step - loss: 0.6517 - accuracy: 0.6195 - val_loss: 0.6089 - val_accuracy: 0.6744
Epoch 6/10
98/98 [==============================] - 20s 204ms/step - loss: 0.6388 - accuracy: 0.6424 - val_loss: 0.6263 - val_accuracy: 0.6585
Epoch 7/10
98/98 [==============================] - 20s 204ms/step - loss: 0.6424 - accuracy: 0.6265 - val_loss: 0.6090 - val_accuracy: 0.6790

In [ ]:
df_train,df_test = load_data(2)
print('The data used will be:')
df_train[['Joined']]

The data used will be:


,Joined
0,"[MK, KD, DK, KQ, QK, KK, KK, KK, KE, ER, RT, T..."
1,"[MS, SW, WS, SG, GL, LL, LH, HG, GL, LN, NT, T..."
2,"[MS, SL, LF, FD, DL, LF, FR, RG, GF, FF, FG, G..."
3,"[MA, AV, VA, AP, PR, RL, LF, FG, GG, GL, LC, C..."
4,"[MS, ST, TI, IQ, QS, SE, ET, TD, DC, CY, YD, D..."
...,...
6265,"[MS, SY, YF, FG, GE, EH, HF, FW, WG, GE, EK, K..."
6266,"[MT, TV, VG, GK, KS, SS, SK, KM, ML, LQ, QH, H..."
6267,"[MN, NN, NL, LS, SF, FS, SE, EL, LC, CC, CL, L..."
6268,"[MA, AP, PE, EI, IN, NL, LP, PG, GP, PM, MS, S..."


In [ ]:
data_join,data_test_join,num_words_join,MAX_SEQUENCE_LENGTH_,MAX_VOCAB_SIZE_ = f.get_seq_data_join(5000,2000,df_train,df_test,pad='pre')

MAX_VOCAB_SIZE is 5000
MAX_SEQUENCE_LENGTH is 2000
max sequence_data length: 8964
min sequence_data length: 105
median sequence_data length: 879
max word index: 400
Found 400 unique tokens.
pre padding.
Shape of data tensor: (6270, 2000)
max sequences_test length: 6124
min sequences_test length: 132
median sequences_test length: 899
pre padding for test seq.
Shape of data_test tensor: (1514, 2000)
num_words is 401


In [ ]:
DROP = 0.3
BATCH_SIZE = 64
EPOCHS = 10
ip = f.transf_model(MAX_SEQUENCE_LENGTH_,num_words_join,EMBEDDING_DIM=5)
x = Dropout(DROP)(ip.output)
x = Dense(128, activation="relu")(x)
x = Dropout(DROP)(x)
outputs = Dense(1, activation="sigmoid")(x)
model1D_CNN_join=Model(ip.input,outputs)

model1D_CNN_join.compile(loss='binary_crossentropy', optimizer=Adam(lr=0.01), metrics=['accuracy'])
model1D_CNN_join.fit(data_join, df_train['label'].values,batch_size= BATCH_SIZE,epochs=EPOCHS, validation_data=(data_test_join,df_test['label'].values))
print(roc_auc_score(df_test['label'].values, model1D_CNN_join.predict(data_test_join)))

Epoch 1/10
98/98 [==============================] - 22s 209ms/step - loss: 0.6846 - accuracy: 0.5606 - val_loss: 0.6248 - val_accuracy: 0.6697
Epoch 2/10
98/98 [==============================] - 20s 205ms/step - loss: 0.6451 - accuracy: 0.6366 - val_loss: 0.6107 - val_accuracy: 0.6797
Epoch 3/10
98/98 [==============================] - 20s 206ms/step - loss: 0.6460 - accuracy: 0.6332 - val_loss: 0.6114 - val_accuracy: 0.6823
Epoch 4/10
98/98 [==============================] - 20s 206ms/step - loss: 0.6337 - accuracy: 0.6483 - val_loss: 0.6008 - val_accuracy: 0.6849
Epoch 5/10
98/98 [==============================] - 20s 205ms/step - loss: 0.6380 - accuracy: 0.6383 - val_loss: 0.5942 - val_accuracy: 0.7041
Epoch 6/10
98/98 [==============================] - 20s 205ms/step - loss: 0.6096 - accuracy: 0.6678 - val_loss: 0.5810 - val_accuracy: 0.7100
Epoch 7/10
98/98 [==============================] - 20s 205ms/step - loss: 0.5967 - accuracy: 0.6860 - val_loss: 0.5769 - val_accuracy: 0.7166

In [ ]:
df_train,df_test = load_data(3)
print('The data used will be:')
df_train[['Joined']]

The data used will be:


,Joined
0,"[MKD, KDK, DKQ, KQK, QKK, KKK, KKK, KKE, KER, ..."
1,"[MSW, SWS, WSG, SGL, GLL, LLH, LHG, HGL, GLN, ..."
2,"[MSL, SLF, LFD, FDL, DLF, LFR, FRG, RGF, GFF, ..."
3,"[MAV, AVA, VAP, APR, PRL, RLF, LFG, FGG, GGL, ..."
4,"[MST, STI, TIQ, IQS, QSE, SET, ETD, TDC, DCY, ..."
...,...
6265,"[MSY, SYF, YFG, FGE, GEH, EHF, HFW, FWG, WGE, ..."
6266,"[MTV, TVG, VGK, GKS, KSS, SSK, SKM, KML, MLQ, ..."
6267,"[MNN, NNL, NLS, LSF, SFS, FSE, SEL, ELC, LCC, ..."
6268,"[MAP, APE, PEI, EIN, INL, NLP, LPG, PGP, GPM, ..."


In [ ]:
data_join,data_test_join,num_words_join,MAX_SEQUENCE_LENGTH_,MAX_VOCAB_SIZE_ = f.get_seq_data_join(5000,2000,df_train,df_test,pad='pre')

MAX_VOCAB_SIZE is 5000
MAX_SEQUENCE_LENGTH is 2000
max sequence_data length: 7989
min sequence_data length: 83
median sequence_data length: 775
max word index: 4999
Found 8000 unique tokens.
pre padding.
Shape of data tensor: (6270, 2000)
max sequences_test length: 5409
min sequences_test length: 116
median sequences_test length: 796
pre padding for test seq.
Shape of data_test tensor: (1514, 2000)
num_words is 5000


In [ ]:
DROP = 0.3
BATCH_SIZE = 64
EPOCHS = 10
ip = f.transf_model(MAX_SEQUENCE_LENGTH_,num_words_join,EMBEDDING_DIM=5)
x = Dropout(DROP)(ip.output)
x = Dense(128, activation="relu")(x)
x = Dropout(DROP)(x)
outputs = Dense(1, activation="sigmoid")(x)
model1D_CNN_join=Model(ip.input,outputs)

model1D_CNN_join.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model1D_CNN_join.fit(data_join, df_train['label'].values,batch_size= BATCH_SIZE,epochs=EPOCHS, validation_data=(data_test_join,df_test['label'].values))
print(roc_auc_score(df_test['label'].values, model1D_CNN_join.predict(data_test_join)))

Epoch 1/10
98/98 [==============================] - 22s 207ms/step - loss: 0.6701 - accuracy: 0.5752 - val_loss: 0.6148 - val_accuracy: 0.6678
Epoch 2/10
98/98 [==============================] - 20s 205ms/step - loss: 0.6351 - accuracy: 0.6405 - val_loss: 0.6012 - val_accuracy: 0.6856
Epoch 3/10
98/98 [==============================] - 20s 205ms/step - loss: 0.6186 - accuracy: 0.6495 - val_loss: 0.5958 - val_accuracy: 0.6942
Epoch 4/10
98/98 [==============================] - 20s 205ms/step - loss: 0.5926 - accuracy: 0.6799 - val_loss: 0.5621 - val_accuracy: 0.7417
Epoch 5/10
98/98 [==============================] - 20s 205ms/step - loss: 0.5466 - accuracy: 0.7287 - val_loss: 0.5297 - val_accuracy: 0.7450
Epoch 6/10
98/98 [==============================] - 20s 205ms/step - loss: 0.4891 - accuracy: 0.7628 - val_loss: 0.5001 - val_accuracy: 0.7695
Epoch 7/10
98/98 [==============================] - 20s 205ms/step - loss: 0.4301 - accuracy: 0.8065 - val_loss: 0.5087 - val_accuracy: 0.7688

In [ ]:
df_train,df_test = load_data(4)
print('The data used will be:')
df_train[['Joined']]

The data used will be:


,Joined
0,"[MKDK, KDKQ, DKQK, KQKK, QKKK, KKKK, KKKE, KKE..."
1,"[MSWS, SWSG, WSGL, SGLL, GLLH, LLHG, LHGL, HGL..."
2,"[MSLF, SLFD, LFDL, FDLF, DLFR, LFRG, FRGF, RGF..."
3,"[MAVA, AVAP, VAPR, APRL, PRLF, RLFG, LFGG, FGG..."
4,"[MSTI, STIQ, TIQS, IQSE, QSET, SETD, ETDC, TDC..."
...,...
6265,"[MSYF, SYFG, YFGE, FGEH, GEHF, EHFW, HFWG, FWG..."
6266,"[MTVG, TVGK, VGKS, GKSS, KSSK, SSKM, SKML, KML..."
6267,"[MNNL, NNLS, NLSF, LSFS, SFSE, FSEL, SELC, ELC..."
6268,"[MAPE, APEI, PEIN, EINL, INLP, NLPG, LPGP, PGP..."


In [ ]:
data_join,data_test_join,num_words_join,MAX_SEQUENCE_LENGTH_,MAX_VOCAB_SIZE_ = f.get_seq_data_join(100000,2000,df_train,df_test,pad='pre')

MAX_VOCAB_SIZE is 100000
MAX_SEQUENCE_LENGTH is 2000
max sequence_data length: 8192
min sequence_data length: 89
median sequence_data length: 805
max word index: 99999
Found 157062 unique tokens.
pre padding.
Shape of data tensor: (6270, 2000)
max sequences_test length: 5586
min sequences_test length: 115
median sequences_test length: 825
pre padding for test seq.
Shape of data_test tensor: (1514, 2000)
num_words is 100000


In [ ]:
DROP = 0.5
BATCH_SIZE = 64
EPOCHS = 10
ip = f.transf_model(MAX_SEQUENCE_LENGTH_,num_words_join,5,DROP)
x = Dropout(DROP)(ip.output)
x = Dense(128, activation="relu")(x)
x = Dropout(DROP)(x)
outputs = Dense(1, activation="sigmoid")(x)
model1D_CNN_join=Model(ip.input,outputs)

model1D_CNN_join.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model1D_CNN_join.fit(data_join, df_train['label'].values,batch_size= BATCH_SIZE,epochs=EPOCHS, validation_data=(data_test_join,df_test['label'].values))
print(roc_auc_score(df_test['label'].values, model1D_CNN_join.predict(data_test_join)))

Epoch 1/10
98/98 [==============================] - 22s 216ms/step - loss: 0.6839 - accuracy: 0.5441 - val_loss: 0.6063 - val_accuracy: 0.6797
Epoch 2/10
98/98 [==============================] - 21s 214ms/step - loss: 0.6104 - accuracy: 0.6645 - val_loss: 0.5800 - val_accuracy: 0.6783
Epoch 3/10
98/98 [==============================] - 21s 213ms/step - loss: 0.4972 - accuracy: 0.7580 - val_loss: 0.5162 - val_accuracy: 0.7556
Epoch 4/10
98/98 [==============================] - 21s 213ms/step - loss: 0.2725 - accuracy: 0.8943 - val_loss: 0.5288 - val_accuracy: 0.7721
Epoch 5/10
98/98 [==============================] - 21s 213ms/step - loss: 0.1587 - accuracy: 0.9430 - val_loss: 0.6726 - val_accuracy: 0.7794
Epoch 6/10
98/98 [==============================] - 21s 213ms/step - loss: 0.1453 - accuracy: 0.9419 - val_loss: 0.5808 - val_accuracy: 0.8078
Epoch 7/10
98/98 [==============================] - 21s 214ms/step - loss: 0.0732 - accuracy: 0.9759 - val_loss: 0.6822 - val_accuracy: 0.8065

In [ ]:
df_train,df_test = load_data(5)
print('The data used will be:')
df_train[['Joined']]

The data used will be:


,Joined
0,"[MKDKQ, KDKQK, DKQKK, KQKKK, QKKKK, KKKKE, KKK..."
1,"[MSWSG, SWSGL, WSGLL, SGLLH, GLLHG, LLHGL, LHG..."
2,"[MSLFD, SLFDL, LFDLF, FDLFR, DLFRG, LFRGF, FRG..."
3,"[MAVAP, AVAPR, VAPRL, APRLF, PRLFG, RLFGG, LFG..."
4,"[MSTIQ, STIQS, TIQSE, IQSET, QSETD, SETDC, ETD..."
...,...
6265,"[MSYFG, SYFGE, YFGEH, FGEHF, GEHFW, EHFWG, HFW..."
6266,"[MTVGK, TVGKS, VGKSS, GKSSK, KSSKM, SSKML, SKM..."
6267,"[MNNLS, NNLSF, NLSFS, LSFSE, SFSEL, FSELC, SEL..."
6268,"[MAPEI, APEIN, PEINL, EINLP, INLPG, NLPGP, LPG..."


In [ ]:
data_join,data_test_join,num_words_join,MAX_SEQUENCE_LENGTH_,MAX_VOCAB_SIZE_ = f.get_seq_data_join(500000,2000,df_train,df_test,pad='pre')

MAX_VOCAB_SIZE is 500000
MAX_SEQUENCE_LENGTH is 2000
max sequence_data length: 5818
min sequence_data length: 29
median sequence_data length: 585
max word index: 499999
Found 1562791 unique tokens.
pre padding.
Shape of data tensor: (6270, 2000)
max sequences_test length: 4978
min sequences_test length: 53
median sequences_test length: 518
pre padding for test seq.
Shape of data_test tensor: (1514, 2000)
num_words is 500000


In [ ]:
DROP = 0.7
BATCH_SIZE = 64
EPOCHS = 10
ip = f.transf_model(MAX_SEQUENCE_LENGTH_,num_words_join,5,DROP)
x = Dropout(DROP)(ip.output)
x = Dense(128, activation="relu")(x)
x = Dropout(DROP)(x)
outputs = Dense(1, activation="sigmoid")(x)
model1D_CNN_join=Model(ip.input,outputs)

model1D_CNN_join.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model1D_CNN_join.fit(data_join, df_train['label'].values,batch_size= BATCH_SIZE,epochs=EPOCHS, validation_data=(data_test_join,df_test['label'].values))
print(roc_auc_score(df_test['label'].values, model1D_CNN_join.predict(data_test_join)))

Epoch 1/10
98/98 [==============================] - 24s 231ms/step - loss: 0.6800 - accuracy: 0.5644 - val_loss: 0.5475 - val_accuracy: 0.7246
Epoch 2/10
98/98 [==============================] - 22s 228ms/step - loss: 0.5415 - accuracy: 0.7430 - val_loss: 0.4716 - val_accuracy: 0.7966
Epoch 3/10
98/98 [==============================] - 22s 229ms/step - loss: 0.3554 - accuracy: 0.8661 - val_loss: 0.5063 - val_accuracy: 0.7794
Epoch 4/10
98/98 [==============================] - 22s 229ms/step - loss: 0.2335 - accuracy: 0.9176 - val_loss: 0.5494 - val_accuracy: 0.8164
Epoch 5/10
98/98 [==============================] - 22s 229ms/step - loss: 0.1170 - accuracy: 0.9603 - val_loss: 0.5706 - val_accuracy: 0.8052
Epoch 6/10
98/98 [==============================] - 22s 229ms/step - loss: 0.0825 - accuracy: 0.9757 - val_loss: 0.6129 - val_accuracy: 0.8190
Epoch 7/10
98/98 [==============================] - 22s 229ms/step - loss: 0.0584 - accuracy: 0.9839 - val_loss: 0.6595 - val_accuracy: 0.8124

In [ ]:
df_train,df_test = load_data(6)
print('The data used will be:')
df_train[['Joined']]

The data used will be:


,Joined
0,"[MKDKQK, KDKQKK, DKQKKK, KQKKKK, QKKKKE, KKKKE..."
1,"[MSWSGL, SWSGLL, WSGLLH, SGLLHG, GLLHGL, LLHGL..."
2,"[MSLFDL, SLFDLF, LFDLFR, FDLFRG, DLFRGF, LFRGF..."
3,"[MAVAPR, AVAPRL, VAPRLF, APRLFG, PRLFGG, RLFGG..."
4,"[MSTIQS, STIQSE, TIQSET, IQSETD, QSETDC, SETDC..."
...,...
6265,"[MSYFGE, SYFGEH, YFGEHF, FGEHFW, GEHFWG, EHFWG..."
6266,"[MTVGKS, TVGKSS, VGKSSK, GKSSKM, KSSKML, SSKML..."
6267,"[MNNLSF, NNLSFS, NLSFSE, LSFSEL, SFSELC, FSELC..."
6268,"[MAPEIN, APEINL, PEINLP, EINLPG, INLPGP, NLPGP..."


In [ ]:
data_join,data_test_join,num_words_join,MAX_SEQUENCE_LENGTH_,MAX_VOCAB_SIZE_ = f.get_seq_data_join(850000,2000,df_train,df_test,pad='pre')

MAX_VOCAB_SIZE is 850000
MAX_SEQUENCE_LENGTH is 2000
max sequence_data length: 6856
min sequence_data length: 5
median sequence_data length: 436
max word index: 849999
Found 3205693 unique tokens.
pre padding.
Shape of data tensor: (6270, 2000)
max sequences_test length: 5558
min sequences_test length: 4
median sequences_test length: 261
pre padding for test seq.
Shape of data_test tensor: (1514, 2000)
num_words is 850000


In [ ]:
DROP = 0.7
BATCH_SIZE = 64
EPOCHS = 10
ip = f.transf_model(MAX_SEQUENCE_LENGTH_,num_words_join,10,DROP)
x = Dropout(DROP)(ip.output)
x = Dense(128, activation="relu")(x)
x = Dropout(DROP)(x)
outputs = Dense(1, activation="sigmoid")(x)
model1D_CNN_join=Model(ip.input,outputs)

model1D_CNN_join.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model1D_CNN_join.fit(data_join, df_train['label'].values,batch_size= BATCH_SIZE,epochs=EPOCHS, validation_data=(data_test_join,df_test['label'].values))
print(roc_auc_score(df_test['label'].values, model1D_CNN_join.predict(data_test_join)))

Epoch 1/10
98/98 [==============================] - 30s 286ms/step - loss: 0.6666 - accuracy: 0.5916 - val_loss: 0.5054 - val_accuracy: 0.7596
Epoch 2/10
98/98 [==============================] - 28s 283ms/step - loss: 0.5003 - accuracy: 0.7750 - val_loss: 0.4660 - val_accuracy: 0.7972
Epoch 3/10
98/98 [==============================] - 28s 282ms/step - loss: 0.3446 - accuracy: 0.8665 - val_loss: 0.4669 - val_accuracy: 0.8118
Epoch 4/10
98/98 [==============================] - 28s 281ms/step - loss: 0.2615 - accuracy: 0.9037 - val_loss: 0.4479 - val_accuracy: 0.8078
Epoch 5/10
98/98 [==============================] - 27s 280ms/step - loss: 0.2343 - accuracy: 0.9101 - val_loss: 0.4989 - val_accuracy: 0.8170
Epoch 6/10
98/98 [==============================] - 28s 282ms/step - loss: 0.1563 - accuracy: 0.9407 - val_loss: 0.4842 - val_accuracy: 0.8177
Epoch 7/10
98/98 [==============================] - 27s 280ms/step - loss: 0.1421 - accuracy: 0.9492 - val_loss: 0.5290 - val_accuracy: 0.8137

###Differential join

In [ ]:
def diff_prot(df,dikt):
  df.Yersinia = df.Yersinia.apply(lambda x: [dikt[char] for char in x])
  df.Joined = [df.loc[row]['Human']+df.loc[row]['Yersinia'] for row in range(df.shape[0])]
  return df



df_train,df_test = load_data(1)
dikt={}
for char in set(df_train.loc[0]['Yersinia']+df_train.loc[0]['Human']):
    dikt[char] = chr(ord(char)+150)
df_test = diff_prot(df_test,dikt)
df_train = diff_prot(df_train,dikt)

print('Yersinia is changed to create seperate words:')
df_train

Yersinia is changed to create seperate words:


,Human,Yersinia,Joined,label
0,"[M, K, D, K, Q, K, K, K, K, E, R, T, W, A, E, ...","[ã, ×, á, ×, é, è, Þ, ä, â, é, ß, Ý, ß, é, ã, ...","[M, K, D, K, Q, K, K, K, K, E, R, T, W, A, E, ...",1
1,"[M, S, W, S, G, L, L, H, G, L, N, T, S, L, T, ...","[ã, ç, Þ, ì, ê, Ý, é, á, è, è, â, ß, Ý, í, ì, ...","[M, S, W, S, G, L, L, H, G, L, N, T, S, L, T, ...",0
2,"[M, S, L, F, D, L, F, R, G, F, F, G, F, P, G, ...","[ã, ×, Û, â, æ, ×, á, è, è, Ü, ê, è, ß, Ú, è, ...","[M, S, L, F, D, L, F, R, G, F, F, G, F, P, G, ...",1
3,"[M, A, V, A, P, R, L, F, G, G, L, C, F, R, F, ...","[ã, è, ß, Ü, ×, ß, é, Ù, é, é, ï, ê, ï, Ù, ï, ...","[M, A, V, A, P, R, L, F, G, G, L, C, F, R, F, ...",0
4,"[M, S, T, I, Q, S, E, T, D, C, Y, D, I, I, E, ...","[ã, é, ï, ×, Ü, æ, Ý, ê, Ü, æ, Ý, è, è, ã, è, ...","[M, S, T, I, Q, S, E, T, D, C, Y, D, I, I, E, ...",0
...,...,...,...,...
6265,"[M, S, Y, F, G, E, H, F, W, G, E, K, N, H, G, ...","[ã, ß, ê, ê, Ú, Ý, ä, é, ×, ì, ×, é, ì, ×, ï, ...","[M, S, Y, F, G, E, H, F, W, G, E, K, N, H, G, ...",1
6266,"[M, T, V, G, K, S, S, K, M, L, Q, H, I, D, Y, ...","[ã, é, ç, æ, æ, Ü, í, ç, ç, á, ê, â, ×, Û, ã, ...","[M, T, V, G, K, S, S, K, M, L, Q, H, I, D, Y, ...",1
6267,"[M, N, N, L, S, F, S, E, L, C, C, L, F, C, C, ...","[ã, é, Û, Ú, è, Þ, ç, ç, è, ç, ç, è, â, á, Û, ...","[M, N, N, L, S, F, S, E, L, C, C, L, F, C, C, ...",0
6268,"[M, A, P, E, I, N, L, P, G, P, M, S, L, I, D, ...","[ã, á, ä, â, é, Ü, ì, ×, Ý, â, ì, ß, â, â, Ý, ...","[M, A, P, E, I, N, L, P, G, P, M, S, L, I, D, ...",1


In [ ]:
data_join,data_test_join,num_words_join,MAX_SEQUENCE_LENGTH_,MAX_VOCAB_SIZE_ = f.get_seq_data_join(5000,2000,df_train,df_test,pad='pre')

MAX_VOCAB_SIZE is 5000
MAX_SEQUENCE_LENGTH is 2000
max sequence_data length: 8966
min sequence_data length: 107
median sequence_data length: 881
max word index: 40
Found 40 unique tokens.
pre padding.
Shape of data tensor: (6270, 2000)
max sequences_test length: 6126
min sequences_test length: 134
median sequences_test length: 901
pre padding for test seq.
Shape of data_test tensor: (1514, 2000)
num_words is 41


In [ ]:
DROP = 0.4
BATCH_SIZE = 64
EPOCHS = 10
ip = f.transf_model(MAX_SEQUENCE_LENGTH_,num_words_join,5,DROP)
x = Dropout(DROP)(ip.output)
x = Dense(128, activation="relu")(x)
x = Dropout(DROP)(x)
outputs = Dense(1, activation="sigmoid")(x)
model1D_CNN_join=Model(ip.input,outputs)

model1D_CNN_join.compile(loss='binary_crossentropy', optimizer=Adam(lr=0.01), metrics=['accuracy'])
model1D_CNN_join.fit(data_join, df_train['label'].values,batch_size= BATCH_SIZE,epochs=EPOCHS, validation_data=(data_test_join,df_test['label'].values))
print(roc_auc_score(df_test['label'].values, model1D_CNN_join.predict(data_test_join)))

Epoch 1/10
98/98 [==============================] - 22s 208ms/step - loss: 0.6813 - accuracy: 0.5485 - val_loss: 0.6131 - val_accuracy: 0.6717
Epoch 2/10
98/98 [==============================] - 20s 205ms/step - loss: 0.6242 - accuracy: 0.6622 - val_loss: 0.5983 - val_accuracy: 0.6823
Epoch 3/10
98/98 [==============================] - 20s 204ms/step - loss: 0.6143 - accuracy: 0.6659 - val_loss: 0.5976 - val_accuracy: 0.6876
Epoch 4/10
98/98 [==============================] - 20s 205ms/step - loss: 0.6101 - accuracy: 0.6734 - val_loss: 0.5935 - val_accuracy: 0.6803
Epoch 5/10
98/98 [==============================] - 20s 205ms/step - loss: 0.6060 - accuracy: 0.6714 - val_loss: 0.5821 - val_accuracy: 0.6869
Epoch 6/10
98/98 [==============================] - 20s 205ms/step - loss: 0.5848 - accuracy: 0.6901 - val_loss: 0.5715 - val_accuracy: 0.7008
Epoch 7/10
98/98 [==============================] - 20s 205ms/step - loss: 0.5802 - accuracy: 0.6801 - val_loss: 0.5596 - val_accuracy: 0.7061

In [ ]:
df_train,df_test = load_data(1)
df_test = diff_prot(df_test,dikt)
df_train = diff_prot(df_train,dikt)
df_train = f.combine_AC(df_train,chunksize=2)
df_test = f.combine_AC(df_test,chunksize=2)
df_train

,Human,Yersinia,Joined,label
0,"[MK, KD, DK, KQ, QK, KK, KK, KK, KE, ER, RT, T...","[ã×, ×á, á×, ×é, éè, èÞ, Þä, äâ, âé, éß, ßÝ, Ý...","[MK, KD, DK, KQ, QK, KK, KK, KK, KE, ER, RT, T...",1
1,"[MS, SW, WS, SG, GL, LL, LH, HG, GL, LN, NT, T...","[ãç, çÞ, Þì, ìê, êÝ, Ýé, éá, áè, èè, èâ, âß, ß...","[MS, SW, WS, SG, GL, LL, LH, HG, GL, LN, NT, T...",0
2,"[MS, SL, LF, FD, DL, LF, FR, RG, GF, FF, FG, G...","[ã×, ×Û, Ûâ, âæ, æ×, ×á, áè, èè, èÜ, Üê, êè, è...","[MS, SL, LF, FD, DL, LF, FR, RG, GF, FF, FG, G...",1
3,"[MA, AV, VA, AP, PR, RL, LF, FG, GG, GL, LC, C...","[ãè, èß, ßÜ, Ü×, ×ß, ßé, éÙ, Ùé, éé, éï, ïê, ê...","[MA, AV, VA, AP, PR, RL, LF, FG, GG, GL, LC, C...",0
4,"[MS, ST, TI, IQ, QS, SE, ET, TD, DC, CY, YD, D...","[ãé, éï, ï×, ×Ü, Üæ, æÝ, Ýê, êÜ, Üæ, æÝ, Ýè, è...","[MS, ST, TI, IQ, QS, SE, ET, TD, DC, CY, YD, D...",0
...,...,...,...,...
6265,"[MS, SY, YF, FG, GE, EH, HF, FW, WG, GE, EK, K...","[ãß, ßê, êê, êÚ, ÚÝ, Ýä, äé, é×, ×ì, ì×, ×é, é...","[MS, SY, YF, FG, GE, EH, HF, FW, WG, GE, EK, K...",1
6266,"[MT, TV, VG, GK, KS, SS, SK, KM, ML, LQ, QH, H...","[ãé, éç, çæ, ææ, æÜ, Üí, íç, çç, çá, áê, êâ, â...","[MT, TV, VG, GK, KS, SS, SK, KM, ML, LQ, QH, H...",1
6267,"[MN, NN, NL, LS, SF, FS, SE, EL, LC, CC, CL, L...","[ãé, éÛ, ÛÚ, Úè, èÞ, Þç, çç, çè, èç, çç, çè, è...","[MN, NN, NL, LS, SF, FS, SE, EL, LC, CC, CL, L...",0
6268,"[MA, AP, PE, EI, IN, NL, LP, PG, GP, PM, MS, S...","[ãá, áä, äâ, âé, éÜ, Üì, ì×, ×Ý, Ýâ, âì, ìß, ß...","[MA, AP, PE, EI, IN, NL, LP, PG, GP, PM, MS, S...",1


In [ ]:
data_join,data_test_join,num_words_join,MAX_SEQUENCE_LENGTH_,MAX_VOCAB_SIZE_ = f.get_seq_data_join(5000,2000,df_train,df_test,pad='pre')

MAX_VOCAB_SIZE is 5000
MAX_SEQUENCE_LENGTH is 2000
max sequence_data length: 8964
min sequence_data length: 105
median sequence_data length: 879
max word index: 800
Found 800 unique tokens.
pre padding.
Shape of data tensor: (6270, 2000)
max sequences_test length: 6124
min sequences_test length: 132
median sequences_test length: 899
pre padding for test seq.
Shape of data_test tensor: (1514, 2000)
num_words is 801


In [ ]:
DROP = 0.4
BATCH_SIZE = 64
EPOCHS = 10
ip = f.transf_model(MAX_SEQUENCE_LENGTH_,num_words_join,5,DROP)
x = Dropout(DROP)(ip.output)
x = Dense(128, activation="relu")(x)
x = Dropout(DROP)(x)
outputs = Dense(1, activation="sigmoid")(x)
model1D_CNN_join=Model(ip.input,outputs)

model1D_CNN_join.compile(loss='binary_crossentropy', optimizer=Adam(lr=0.01), metrics=['accuracy'])
model1D_CNN_join.fit(data_join, df_train['label'].values,batch_size= BATCH_SIZE,epochs=EPOCHS, validation_data=(data_test_join,df_test['label'].values))
print(roc_auc_score(df_test['label'].values, model1D_CNN_join.predict(data_test_join)))

Epoch 1/10
98/98 [==============================] - 22s 209ms/step - loss: 0.6767 - accuracy: 0.5682 - val_loss: 0.5960 - val_accuracy: 0.6968
Epoch 2/10
98/98 [==============================] - 20s 206ms/step - loss: 0.6180 - accuracy: 0.6703 - val_loss: 0.5849 - val_accuracy: 0.6988
Epoch 3/10
98/98 [==============================] - 20s 206ms/step - loss: 0.6024 - accuracy: 0.6790 - val_loss: 0.5843 - val_accuracy: 0.7081
Epoch 4/10
98/98 [==============================] - 20s 205ms/step - loss: 0.5860 - accuracy: 0.6921 - val_loss: 0.5540 - val_accuracy: 0.7061
Epoch 5/10
98/98 [==============================] - 20s 205ms/step - loss: 0.5685 - accuracy: 0.7006 - val_loss: 0.5508 - val_accuracy: 0.7034
Epoch 6/10
98/98 [==============================] - 20s 205ms/step - loss: 0.5330 - accuracy: 0.7329 - val_loss: 0.5441 - val_accuracy: 0.7153
Epoch 7/10
98/98 [==============================] - 20s 206ms/step - loss: 0.5171 - accuracy: 0.7488 - val_loss: 0.5696 - val_accuracy: 0.7133

In [ ]:
df_train = f.combine_AC(df_train,chunksize=3)
df_test = f.combine_AC(df_test,chunksize=3)
df_test

,Human,Yersinia,Joined,label
0,"[MYV, YVT, VTM, TMM, MMM, MMT, MTD, TDQ, DQI, ...","[ãêÝ, êÝæ, Ýæç, æçç, ççÝ, çÝá, Ýáß, áßâ, ßââ, ...","[MYV, YVT, VTM, TMM, MMM, MMT, MTD, TDQ, DQI, ...",1
1,"[MED, EDH, DHQ, HQH, QHV, HVP, VPI, PID, IDI, ...","[ãáä, áäè, äèê, èêâ, êâÝ, âÝé, Ýéß, éßÜ, ßÜß, ...","[MED, EDH, DHQ, HQH, QHV, HVP, VPI, PID, IDI, ...",1
2,"[MVL, VLG, LGK, GKV, KVK, VKS, KSL, SLT, LTI, ...","[ãßß, ßßì, ßìê, ìêÝ, êÝÝ, ÝÝ×, Ý×Ý, ×ÝÜ, ÝÜß, ...","[MVL, VLG, LGK, GKV, KVK, VKS, KSL, SLT, LTI, ...",1
3,"[MEH, EHK, HKE, KEV, EVV, VVL, VLL, LLL, LLL, ...","[ãâß, âßá, ßáâ, áââ, ââê, âêá, êáì, áìÜ, ìÜÝ, ...","[MEH, EHK, HKE, KEV, EVV, VVL, VLL, LLL, LLL, ...",1
4,"[MAP, APR, PRT, RTL, TLW, LWS, WSC, SCY, CYL, ...","[ãêä, êäê, äêé, êéï, éïß, ïßá, ßáä, áä×, ä×Û, ...","[MAP, APR, PRT, RTL, TLW, LWS, WSC, SCY, CYL, ...",0
...,...,...,...,...
1509,"[MGT, GTP, TPA, PAS, ASG, SGR, GRK, RKR, KRT, ...","[ãßá, ßáâ, áâÝ, âÝß, Ýßì, ßìã, ìãÚ, ãÚæ, Úæß, ...","[MGT, GTP, TPA, PAS, ASG, SGR, GRK, RKR, KRT, ...",1
1510,"[MRS, RSA, SAA, AAA, AAA, AAP, APR, PRS, RSP, ...","[ãèé, èéÙ, éÙâ, Ùâé, âéÝ, éÝê, Ýêè, êèÙ, èÙì, ...","[MRS, RSA, SAA, AAA, AAA, AAP, APR, PRS, RSP, ...",1
1511,"[MLL, LLR, LRL, RLL, LLL, LLA, LAW, AWA, WAA, ...","[ãèì, èìÛ, ìÛß, ÛßÝ, ßÝä, Ýäã, äãé, ãéÝ, éÝÚ, ...","[MLL, LLR, LRL, RLL, LLL, LLA, LAW, AWA, WAA, ...",0
1512,"[MTL, TLE, LEE, EEL, ELV, LVA, VAC, ACD, CDN, ...","[ãèâ, èâÜ, âÜ×, Ü×ç, ×çâ, çâÝ, âÝí, Ýíï, íïÜ, ...","[MTL, TLE, LEE, EEL, ELV, LVA, VAC, ACD, CDN, ...",0


In [ ]:
data_join,data_test_join,num_words_join,MAX_SEQUENCE_LENGTH_,MAX_VOCAB_SIZE_ = f.get_seq_data_join(10000,2000,df_train,df_test,pad='pre')

MAX_VOCAB_SIZE is 10000
MAX_SEQUENCE_LENGTH is 2000
max sequence_data length: 8373
min sequence_data length: 91
median sequence_data length: 782
max word index: 9999
Found 15998 unique tokens.
pre padding.
Shape of data tensor: (6270, 2000)
max sequences_test length: 5717
min sequences_test length: 113
median sequences_test length: 800
pre padding for test seq.
Shape of data_test tensor: (1514, 2000)
num_words is 10000


In [ ]:
DROP = 0.3
BATCH_SIZE = 64
EPOCHS = 10
ip = f.transf_model(MAX_SEQUENCE_LENGTH_,num_words_join,EMBEDDING_DIM=5)
x = Dropout(DROP)(ip.output)
x = Dense(128, activation="relu")(x)
x = Dropout(DROP)(x)
outputs = Dense(1, activation="sigmoid")(x)
model1D_CNN_join=Model(ip.input,outputs)

model1D_CNN_join.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model1D_CNN_join.fit(data_join, df_train['label'].values,batch_size= BATCH_SIZE,epochs=EPOCHS, validation_data=(data_test_join,df_test['label'].values))
print(roc_auc_score(df_test['label'].values, model1D_CNN_join.predict(data_test_join)))

Epoch 1/10
98/98 [==============================] - 22s 207ms/step - loss: 0.6753 - accuracy: 0.5541 - val_loss: 0.5941 - val_accuracy: 0.6849
Epoch 2/10
98/98 [==============================] - 20s 204ms/step - loss: 0.5964 - accuracy: 0.6845 - val_loss: 0.5585 - val_accuracy: 0.7160
Epoch 3/10
98/98 [==============================] - 20s 204ms/step - loss: 0.5736 - accuracy: 0.7062 - val_loss: 0.5207 - val_accuracy: 0.7510
Epoch 4/10
98/98 [==============================] - 20s 205ms/step - loss: 0.4831 - accuracy: 0.7739 - val_loss: 0.4976 - val_accuracy: 0.7675
Epoch 5/10
98/98 [==============================] - 20s 204ms/step - loss: 0.3678 - accuracy: 0.8470 - val_loss: 0.4505 - val_accuracy: 0.7933
Epoch 6/10
98/98 [==============================] - 20s 204ms/step - loss: 0.2811 - accuracy: 0.8890 - val_loss: 0.5249 - val_accuracy: 0.7662
Epoch 7/10
98/98 [==============================] - 20s 205ms/step - loss: 0.2363 - accuracy: 0.9064 - val_loss: 0.5303 - val_accuracy: 0.7906

In [ ]:
df_train,df_test = load_data(1)
df_test = diff_prot(df_test,dikt)
df_train = diff_prot(df_train,dikt)
df_train = f.combine_AC(df_train,chunksize=4)
df_test = f.combine_AC(df_test,chunksize=4)
df_test

,Human,Yersinia,Joined,label
0,"[MYVT, YVTM, VTMM, TMMM, MMMT, MMTD, MTDQ, TDQ...","[ãêÝæ, êÝæç, Ýæçç, æççÝ, ççÝá, çÝáß, Ýáßâ, áßâ...","[MYVT, YVTM, VTMM, TMMM, MMMT, MMTD, MTDQ, TDQ...",1
1,"[MEDH, EDHQ, DHQH, HQHV, QHVP, HVPI, VPID, PID...","[ãáäè, áäèê, äèêâ, èêâÝ, êâÝé, âÝéß, ÝéßÜ, éßÜ...","[MEDH, EDHQ, DHQH, HQHV, QHVP, HVPI, VPID, PID...",1
2,"[MVLG, VLGK, LGKV, GKVK, KVKS, VKSL, KSLT, SLT...","[ãßßì, ßßìê, ßìêÝ, ìêÝÝ, êÝÝ×, ÝÝ×Ý, Ý×ÝÜ, ×ÝÜ...","[MVLG, VLGK, LGKV, GKVK, KVKS, VKSL, KSLT, SLT...",1
3,"[MEHK, EHKE, HKEV, KEVV, EVVL, VVLL, VLLL, LLL...","[ãâßá, âßáâ, ßáââ, áââê, ââêá, âêáì, êáìÜ, áìÜ...","[MEHK, EHKE, HKEV, KEVV, EVVL, VVLL, VLLL, LLL...",1
4,"[MAPR, APRT, PRTL, RTLW, TLWS, LWSC, WSCY, SCY...","[ãêäê, êäêé, äêéï, êéïß, éïßá, ïßáä, ßáä×, áä×...","[MAPR, APRT, PRTL, RTLW, TLWS, LWSC, WSCY, SCY...",0
...,...,...,...,...
1509,"[MGTP, GTPA, TPAS, PASG, ASGR, SGRK, GRKR, RKR...","[ãßáâ, ßáâÝ, áâÝß, âÝßì, Ýßìã, ßìãÚ, ìãÚæ, ãÚæ...","[MGTP, GTPA, TPAS, PASG, ASGR, SGRK, GRKR, RKR...",1
1510,"[MRSA, RSAA, SAAA, AAAA, AAAP, AAPR, APRS, PRS...","[ãèéÙ, èéÙâ, éÙâé, ÙâéÝ, âéÝê, éÝêè, ÝêèÙ, êèÙ...","[MRSA, RSAA, SAAA, AAAA, AAAP, AAPR, APRS, PRS...",1
1511,"[MLLR, LLRL, LRLL, RLLL, LLLA, LLAW, LAWA, AWA...","[ãèìÛ, èìÛß, ìÛßÝ, ÛßÝä, ßÝäã, Ýäãé, äãéÝ, ãéÝ...","[MLLR, LLRL, LRLL, RLLL, LLLA, LLAW, LAWA, AWA...",0
1512,"[MTLE, TLEE, LEEL, EELV, ELVA, LVAC, VACD, ACD...","[ãèâÜ, èâÜ×, âÜ×ç, Ü×çâ, ×çâÝ, çâÝí, âÝíï, Ýíï...","[MTLE, TLEE, LEEL, EELV, ELVA, LVAC, VACD, ACD...",0


In [ ]:
data_join,data_test_join,num_words_join,MAX_SEQUENCE_LENGTH_,MAX_VOCAB_SIZE_ = f.get_seq_data_join(200000,2000,df_train,df_test,pad='pre')

MAX_VOCAB_SIZE is 200000
MAX_SEQUENCE_LENGTH is 2000
max sequence_data length: 8527
min sequence_data length: 94
median sequence_data length: 821
max word index: 199999
Found 290131 unique tokens.
pre padding.
Shape of data tensor: (6270, 2000)
max sequences_test length: 5854
min sequences_test length: 110
median sequences_test length: 836
pre padding for test seq.
Shape of data_test tensor: (1514, 2000)
num_words is 200000


In [ ]:
DROP = 0.7
BATCH_SIZE = 64
EPOCHS = 10
ip = f.transf_model(MAX_SEQUENCE_LENGTH_,num_words_join,5,DROP)
x = Dropout(DROP)(ip.output)
x = Dense(128, activation="relu")(x)
x = Dropout(DROP)(x)
outputs = Dense(1, activation="sigmoid")(x)
model1D_CNN_join=Model(ip.input,outputs)

model1D_CNN_join.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model1D_CNN_join.fit(data_join, df_train['label'].values,batch_size= BATCH_SIZE,epochs=EPOCHS, validation_data=(data_test_join,df_test['label'].values))
print(roc_auc_score(df_test['label'].values, model1D_CNN_join.predict(data_test_join)))

Epoch 1/10
98/98 [==============================] - 23s 220ms/step - loss: 0.6864 - accuracy: 0.5489 - val_loss: 0.6107 - val_accuracy: 0.6651
Epoch 2/10
98/98 [==============================] - 21s 217ms/step - loss: 0.6317 - accuracy: 0.6409 - val_loss: 0.5704 - val_accuracy: 0.7120
Epoch 3/10
98/98 [==============================] - 21s 217ms/step - loss: 0.5119 - accuracy: 0.7550 - val_loss: 0.4522 - val_accuracy: 0.7959
Epoch 4/10
98/98 [==============================] - 21s 217ms/step - loss: 0.2677 - accuracy: 0.9000 - val_loss: 0.4775 - val_accuracy: 0.8012
Epoch 5/10
98/98 [==============================] - 21s 217ms/step - loss: 0.1202 - accuracy: 0.9663 - val_loss: 0.6004 - val_accuracy: 0.7814
Epoch 6/10
98/98 [==============================] - 21s 217ms/step - loss: 0.0802 - accuracy: 0.9760 - val_loss: 0.6200 - val_accuracy: 0.8091
Epoch 7/10
98/98 [==============================] - 21s 217ms/step - loss: 0.0513 - accuracy: 0.9880 - val_loss: 0.7484 - val_accuracy: 0.8118

In [ ]:
df_train,df_test = load_data(1)
df_test = diff_prot(df_test,dikt)
df_train = diff_prot(df_train,dikt)
df_train = f.combine_AC(df_train,chunksize=5)
df_test = f.combine_AC(df_test,chunksize=5)
df_test

,Human,Yersinia,Joined,label
0,"[MYVTM, YVTMM, VTMMM, TMMMT, MMMTD, MMTDQ, MTD...","[ãêÝæç, êÝæçç, ÝæççÝ, æççÝá, ççÝáß, çÝáßâ, Ýáß...","[MYVTM, YVTMM, VTMMM, TMMMT, MMMTD, MMTDQ, MTD...",1
1,"[MEDHQ, EDHQH, DHQHV, HQHVP, QHVPI, HVPID, VPI...","[ãáäèê, áäèêâ, äèêâÝ, èêâÝé, êâÝéß, âÝéßÜ, Ýéß...","[MEDHQ, EDHQH, DHQHV, HQHVP, QHVPI, HVPID, VPI...",1
2,"[MVLGK, VLGKV, LGKVK, GKVKS, KVKSL, VKSLT, KSL...","[ãßßìê, ßßìêÝ, ßìêÝÝ, ìêÝÝ×, êÝÝ×Ý, ÝÝ×ÝÜ, Ý×Ý...","[MVLGK, VLGKV, LGKVK, GKVKS, KVKSL, VKSLT, KSL...",1
3,"[MEHKE, EHKEV, HKEVV, KEVVL, EVVLL, VVLLL, VLL...","[ãâßáâ, âßáââ, ßáââê, áââêá, ââêáì, âêáìÜ, êáì...","[MEHKE, EHKEV, HKEVV, KEVVL, EVVLL, VVLLL, VLL...",1
4,"[MAPRT, APRTL, PRTLW, RTLWS, TLWSC, LWSCY, WSC...","[ãêäêé, êäêéï, äêéïß, êéïßá, éïßáä, ïßáä×, ßáä...","[MAPRT, APRTL, PRTLW, RTLWS, TLWSC, LWSCY, WSC...",0
...,...,...,...,...
1509,"[MGTPA, GTPAS, TPASG, PASGR, ASGRK, SGRKR, GRK...","[ãßáâÝ, ßáâÝß, áâÝßì, âÝßìã, ÝßìãÚ, ßìãÚæ, ìãÚ...","[MGTPA, GTPAS, TPASG, PASGR, ASGRK, SGRKR, GRK...",1
1510,"[MRSAA, RSAAA, SAAAA, AAAAP, AAAPR, AAPRS, APR...","[ãèéÙâ, èéÙâé, éÙâéÝ, ÙâéÝê, âéÝêè, éÝêèÙ, Ýêè...","[MRSAA, RSAAA, SAAAA, AAAAP, AAAPR, AAPRS, APR...",1
1511,"[MLLRL, LLRLL, LRLLL, RLLLA, LLLAW, LLAWA, LAW...","[ãèìÛß, èìÛßÝ, ìÛßÝä, ÛßÝäã, ßÝäãé, ÝäãéÝ, äãé...","[MLLRL, LLRLL, LRLLL, RLLLA, LLLAW, LLAWA, LAW...",0
1512,"[MTLEE, TLEEL, LEELV, EELVA, ELVAC, LVACD, VAC...","[ãèâÜ×, èâÜ×ç, âÜ×çâ, Ü×çâÝ, ×çâÝí, çâÝíï, âÝí...","[MTLEE, TLEEL, LEELV, EELVA, ELVAC, LVACD, VAC...",0


In [ ]:
data_join,data_test_join,num_words_join,MAX_SEQUENCE_LENGTH_,MAX_VOCAB_SIZE_ = f.get_seq_data_join(500000,2000,df_train,df_test,pad='pre')

MAX_VOCAB_SIZE is 500000
MAX_SEQUENCE_LENGTH is 2000
max sequence_data length: 5818
min sequence_data length: 16
median sequence_data length: 481
max word index: 499999
Found 1969676 unique tokens.
pre padding.
Shape of data tensor: (6270, 2000)
max sequences_test length: 4978
min sequences_test length: 29
median sequences_test length: 374
pre padding for test seq.
Shape of data_test tensor: (1514, 2000)
num_words is 500000


In [ ]:
DROP = 0.8
BATCH_SIZE = 64
EPOCHS = 10
ip = f.transf_model(MAX_SEQUENCE_LENGTH_,num_words_join,5,DROP)
x = Dropout(DROP)(ip.output)
x = Dense(128, activation="relu")(x)
x = Dropout(DROP)(x)
outputs = Dense(1, activation="sigmoid")(x)
model1D_CNN_join=Model(ip.input,outputs)

model1D_CNN_join.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model1D_CNN_join.fit(data_join, df_train['label'].values,batch_size= BATCH_SIZE,epochs=EPOCHS, validation_data=(data_test_join,df_test['label'].values))
print(roc_auc_score(df_test['label'].values, model1D_CNN_join.predict(data_test_join)))

Epoch 1/10
98/98 [==============================] - 24s 232ms/step - loss: 0.7452 - accuracy: 0.5082 - val_loss: 0.6779 - val_accuracy: 0.7305
Epoch 2/10
98/98 [==============================] - 22s 229ms/step - loss: 0.6527 - accuracy: 0.6016 - val_loss: 0.5289 - val_accuracy: 0.7371
Epoch 3/10
98/98 [==============================] - 22s 229ms/step - loss: 0.5218 - accuracy: 0.7723 - val_loss: 0.4577 - val_accuracy: 0.7834
Epoch 4/10
98/98 [==============================] - 22s 229ms/step - loss: 0.3664 - accuracy: 0.8683 - val_loss: 0.4671 - val_accuracy: 0.8038
Epoch 5/10
98/98 [==============================] - 22s 228ms/step - loss: 0.2304 - accuracy: 0.9249 - val_loss: 0.5829 - val_accuracy: 0.7847
Epoch 6/10
98/98 [==============================] - 22s 229ms/step - loss: 0.1740 - accuracy: 0.9466 - val_loss: 0.4749 - val_accuracy: 0.8230
Epoch 7/10
98/98 [==============================] - 22s 228ms/step - loss: 0.1381 - accuracy: 0.9664 - val_loss: 0.7306 - val_accuracy: 0.7926